In [1]:
import pandas as pd
import plotly.express as px
from src.wrapper import create_data, simulate, read_accuracy, anonimize, utility

  ## 1. rész
  - Több féle perturbáció
  - 1 féle deanonimizáció
  - accuracy mennyi
  - scoreboard

In [2]:
# háttérismeret és anonimizálandó adat generálása
# Erős támadó
print("Háttérismeret generálása...\n")
create_data('enron', 2000, 'exp1', 1, 'ns09', 0.8, 0.8, verbose=True)


Háttérismeret generálása...

@ 2021-06-27 15:32:07.97
Creating test data for: enron [exp1]
	(Nodes: 36692, edges: 183831)
	Export params: exports=1, export_size=2000
	Perturb. params: algo=ns09, variants=1, alpha_v=0.8, alpha_e=0.8
	Export: ./output/enron_2.0k_exp1/SimuData/e0.tgf, nodes: 2002, edges: 34579
		Variant: v0
			g_src nodes: 1788, edges: 25605
			g_tar nodes: 1744, edges: 24282
			node overlap: 1541



In [3]:
# Deanonimizáció
# megfigyelhetjük, hogy itt jelentősen sikeresebb a deanon, mikor jobb a háttérismeret
print("Deanonimizációs algoritmus futtatása\n")
simulate('enron', 2000, 'exp1', 'ns09', 'random.25', 50, 0.01, verbose=True)


Deanonimizációs algoritmus futtatása

@ 2021-06-27 15:32:10.963
Simulating on data derived from: enron [exp1]
	Parameters: algo=ns09, num_rounds=1, seed_type=random.25, seed_count=50
	Parameters of ns09: theta=0.01
	Exports: 1, perturbed: 1
Simulating: e0_v0
	g_src nodes: 1788, edges: 25605
	g_tar nodes: 1744, edges: 24282
	node overlap: 1541
	#0
	    50 seed nodes (from top 25%)
	    Propagate: [+++++]
	    5 rounds, 6.85 sec(s) 
	    TP=80.47%, FP=1.3%, N/A=18.23%, FP/TP=1.61%
	    Total error (w/mappings out of GT): 6.36%)



In [4]:
# Tamadás eredényessége
TPR, _, FPR, _ = read_accuracy('enron', 2000, 'exp1', 'ns09')['avg']
print('True positive rate: {:.2f}\nFalse positive rate: {:.2f}'.format(TPR, FPR))


True positive rate: 80.47
False positive rate: 1.30


In [5]:
print('Közepes támadó modell futtatása')
create_data('enron', 2000, 'exp2', 1, 'ns09', 0.5, 0.75)
simulate('enron', 2000, 'exp2', 'ns09', 'random.25', 50, 0.01)

print('Gyenge támadó modell futtatása')
create_data('enron', 2000, 'exp3', 1, 'ns09', 0.4, 0.6)
simulate('enron', 2000, 'exp3', 'ns09', 'random.25', 50, 0.01)

print('Eredmények beolvasása')
df = pd.DataFrame(columns=['experiment', 'deanon', 'TPR', 'FPR'])
for i in range(3):
	experiment = 'exp'+str(i+1)
	TPR, _, FPR, _ = read_accuracy('enron', 2000, experiment, 'ns09')['avg']
	df = df.append({'experiment':experiment, 'deanon':'ns09', 'TPR':TPR, 'FPR':FPR}, ignore_index=True)
df


Közepes támadó modell futtatása
Gyenge támadó modell futtatása
Eredmények beolvasása


,experiment,deanon,TPR,FPR
0,exp1,ns09,80.467229,1.297859
1,exp2,ns09,63.868066,6.671664
2,exp3,ns09,8.776844,5.695612


In [6]:
# Eredmények ábrázolása a score boardon
fig = px.scatter(df, x='FPR', y='TPR', 
    color='experiment', 
    range_y=[0,100],
    labels={
        "FPR": "Incorrect matches (FPR)",
        "TPR": "Correct matches (TPR)",
    },
    title="Scoreboard: Correct vs Incorrect matches")
fig.show()


  ## 2. rész
  - egy perturb
  - több deanon összehasonlítása (nar, blb, KL)
  - accuracy mennyi
  - scoreboard

In [7]:
# paraméterek
experiment = '2_exp'
network = 'enron'
size = 2000


In [8]:
# új háttérismeret generálása
create_data(network, size, experiment, 1, 'ns09', 0.8, 0.8)


In [9]:
# Három deanon algoritmus (ki lehet fejteni szövegesen)
print('ns09 deanonimizáció futtatása')
simulate(network, size, experiment, 'ns09', 'random.25', 50, 0.01)
# ez a blb, mit tud?

ns09 deanonimizáció futtatása


In [10]:
print('blb deanonimizáció futtatása')
simulate(network, size, experiment, 'blb', 'random.25', 50, '0.1,0.5')

blb deanonimizáció futtatása


In [11]:
print('KL deanonimizáció futtatása')
simulate(network, size, experiment, 'KL', 'random.25', 50, 100)


KL deanonimizáció futtatása


In [12]:
print('pontosságok beolvasása')
df = pd.DataFrame(columns=['experiment', 'anon', 'deanon', 'TPR', 'FPR'])
for deanon in ['ns09', 'blb', 'KL']:
	TPR, _, FPR, _ = read_accuracy(network, size, experiment, deanon)['avg']
	df = df.append({'experiment':experiment, 'anon':'none', 'deanon':deanon, 'TPR':TPR, 'FPR':FPR}, ignore_index=True)
df

pontosságok beolvasása


,experiment,anon,deanon,TPR,FPR
0,2_exp,none,ns09,77.700348,1.324042
1,2_exp,none,blb,79.233449,1.254355
2,2_exp,none,KL,78.257840,38.397213


In [13]:
fig = px.scatter(df, x='FPR', y='TPR', 
    symbol='deanon', 
    range_y=[0,100],
    labels={
        "FPR": "Incorrect matches (FPR)",
        "TPR": "Correct matches (TPR)",
    },
    title="Scoreboard: Correct vs Incorrect matches")
fig.show()

 3. rész
 1 perturn, 3 anon, 1 deanon, utility measure

In [18]:
# paraméterek
experiment = '3_exp'
deanon = 'ns09'
network = 'enron'
size = 2000 

print("Háttérismeret generálása")
create_data(network, size, experiment, 1, 'ns09', 0.5, 0.75)

# define parameters
params = {
    'sw': 0.2,
    'kda': 5,
    'dp': 50
}

# results will be sroted in this df
df = pd.DataFrame(columns=['experiment', 'anon', 'deanon', 'TPR', 'FPR', 'utility'])

for anon in ['none', 'sw', 'kda', 'dp']:
    # anonimize graph
    if anon != 'none':
        print("Anonimizálás {} módszerrel".format(anon))
        anonimize(network, size, experiment, anon, params[anon])

    # calculate utility loss
    util = utility(network, size, experiment, 'lcc')

    # run deanon algo
    simulate(network, size, experiment, deanon, 'random.25', 50, 0.01)

    # calculate results
    TPR, _, FPR, _ = read_accuracy(network, size, experiment, deanon)['avg']
    df = df.append({'experiment':experiment, 'anon':anon, 'deanon':deanon, 'TPR':TPR, 'FPR':FPR, 'utility':util}, ignore_index=True)
print("Eredmények értékelése")
df


Háttérismeret generálása
Anonimizálás sw módszerrel
Anonimizálás kda módszerrel
Anonimizálás dp módszerrel
Eredmények értékelése


,experiment,anon,deanon,TPR,FPR,utility
0,3_exp,none,ns09,60.215054,9.293395,1.000000
1,3_exp,sw,ns09,55.990783,10.368664,1.000000
2,3_exp,kda,ns09,49.846390,13.210445,0.986713
3,3_exp,dp,ns09,51.075269,14.516129,0.904360


In [19]:
fig = px.scatter(df, x='FPR', y='TPR', 
    color='anon',
    range_y=[0,100],
    labels={
        "FPR": "Incorrect matches (FPR)",
        "TPR": "Correct matches (TPR)",
    },
    title="Scoreboard: Correct vs Incorrect matches")
fig.show()

 4. rész
 - 1 perturb
 - 3 anon
 - 3 deanon
 - scoreboard
 utility

In [20]:
# paraméterek
network = 'enron'
size = 1000
nseed = 50


In [21]:
# define parameters
params = {
    # anon
    'sw': 0.1,
    'kda': 50,
    'dp': 50,
    # deanon
    'ns09': 0.01,
    'KL': 100,
    'blb': '0.1,0.5',
    # util
    'inf': 5
}

# Háttérismeret generálása
print('Gyenge támadó feltételezve')
create_data(network, size, "gyenge", 1, 'ns09', 0.4, 0.6)

print('Erős támadó feltételezve')
create_data(network, size, "eros", 1, 'ns09', 0.8, 0.8)

# eredmények gyűjtéséhez létrehozott táblázat
df = df_util = pd.DataFrame(columns=['experiment', 'anon', 'deanon', 'TPR', 'FPR'])

# Egyes kombilációk futtatása
for experiment in ['gyenge', 'eros']:
    print('\n'+experiment)
    for anon in ['none', 'sw', 'kda', 'dp']:
        print(anon)
        # anonimize graph
        if (anon != 'none'):
            anonimize(network, size, experiment, anon, params[anon])
        
            # calculate utility loss for anonimization method
            # for util in ['inf', 'deg', 'lcc']:
            #     param = params.get(util, None)
            #     value = utility(network, size, experiment, util, param)
            #     df_util = df_util.append({'anon':anon, 'value':value, 'util': util}, ignore_index=True)

        for deanon in ['ns09', 'KL', 'blb']:
            print('\t'+deanon)
            # run deanon algos
            simulate(network, size, experiment, deanon, 'random.25', nseed, params[deanon])
            # calculate results
            TPR, _, FPR, _ = read_accuracy(network, size, experiment, deanon)['avg']
            df = df.append({'experiment':experiment, 'anon':anon, 'deanon':deanon, 'TPR':TPR, 'FPR':FPR}, ignore_index=True)
df


Gyenge támadó feltételezve
Erős támadó feltételezve

gyenge
none
	ns09
	KL
	blb
sw
	ns09
	KL
	blb
kda
	ns09
	KL
	blb
dp
	ns09
	KL
	blb

eros
none
	ns09
	KL
	blb
sw
	ns09
	KL
	blb
kda
	ns09
	KL
	blb
dp
	ns09
	KL
	blb


,experiment,anon,deanon,TPR,FPR
0,gyenge,none,ns09,23.791103,19.729207
1,gyenge,none,KL,31.528046,105.802708
2,gyenge,none,blb,44.680851,19.148936
3,gyenge,sw,ns09,31.141199,20.116054
4,gyenge,sw,KL,31.528046,105.802708
5,gyenge,sw,blb,49.516441,18.762089
6,gyenge,kda,ns09,13.539652,16.827853
7,gyenge,kda,KL,13.539652,123.791103
8,gyenge,kda,blb,13.346228,11.025145
9,gyenge,dp,ns09,18.955513,26.692456


In [22]:
# Scoreborad
data = df[df['experiment']=='gyenge']
fig = px.scatter(data, x='FPR', y='TPR', 
    color='deanon',
    symbol='anon',
    range_y=[0,100],
    labels={
        "FPR": "Incorrect matches (FPR)",
        "TPR": "Correct matches (TPR)",
    },
    title="Gyenge támadó modellt feltételezve")
fig.show()

In [23]:
# Scoreborad
data = df[df['experiment']=='eros']
fig = px.scatter(data, x='FPR', y='TPR', 
    color='deanon',
    symbol='anon',
    range_y=[0,100],
    labels={
        "FPR": "Incorrect matches (FPR)",
        "TPR": "Correct matches (TPR)",
    },
    title="Erős támadó modellt feltételezve")
fig.show()